In [13]:
from ipyleaflet import Map, Polyline, Marker, AntPath, basemaps, Heatmap
import trajet

In [20]:
ptA = (45.786839, 4.879130)
ptB = (45.782746, 4.878132)
ptC = (45.764346, 4.863172)
heat_map, initial_path, final_path = trajet.getBestPath(ptA, ptC, 0.03, 0.5)

perfect path
2 itterations to find this path


In [21]:
## Add Path

line_initial_path = Polyline(
    locations=[
        initial_path
    ],
    color="#3373FF" ,
    opacity= 0.5,
    fill=False
)

final_path_layer = AntPath(
    locations=[
       final_path
    ],
    dash_array=[1, 10],
    delay=1000,
    color='#45896A',
    pulse_color='#458948'
)
# En blanc Map(basemap=basemaps.CartoDB.Positron, center=center, zoom=zoom)
m = Map(basemap=basemaps.CartoDB.DarkMatter, center=(45.757747, 4.850580),
    zoom=11)
m.add_layer(Marker(location=ptA, draggable=False, opacity = 0.6))
m.add_layer(Marker(location=ptB, draggable=False, opacity = 0.6))
m.add_layer(Marker(location=ptC, draggable=False, opacity = 0.6))

m.add_layer(line_initial_path)
m.add_layer(final_path_layer)

m

Map(center=[45.757747, 4.85058], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [4]:
# add heatmap 
locations = [
    [uniform(-80, 80), uniform(-180, 180), uniform(0, 1000)] # lat, lng, intensity 
    for i in range(1000)
]
heat = Heatmap(locations=locations, radius=20, blur=10)
m.add_layer(heat)

m


NameError: name 'uniform' is not defined